In [2]:
#pip install requests
#pip install bs4

In [228]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import csv

In [37]:
# Найдём все id-шники депутатов, которые есть в VIII созыве
legislators_soup = soup.find_all('a', class_="person__image-wrapper person__image-wrapper--s")

In [227]:
for i in range(len(legislators_soup)):
    # Находим и парсим страницу депутата по полученному айдишнику
    query = legislators_soup[i].get('href')
    r = requests.get(f'http://duma.gov.ru{query}')
    htlmtext = r.text
    legislator_soup = BeautifulSoup(htlmtext, 'html.parser')
    
    # Парсим имя депутата
    name = legislator_soup.find(itemprop="name").get_text(strip='True').split()
    # Имя депутата приводим к формату, в котором скачиваются законопреокты с sozd.duma
    name = name[1][0] + '.' + name[2][0] + '.' + name[0]

    # Парсим название фракции, отбрасываем кавычки, оставляем название заглавными
    faction = legislator_soup.find(itemprop="worksFor").get_text(strip='True')
    faction = faction.strip('«»')
    
    # Парсим название комитета. Комитет, судя по всему, только один
    comittee = legislator_soup.find(class_="link link--underline link--external")
    if comittee is not None:
        comittee = comittee.get_text(strip='True')
        comittee = comittee.split(' (')[0]
    
    # Парсим название комиссий. Они спрятаны чуть хитрее, в параметре dd. 
    # Их может больше чем две, так что делаем массив
    comissions = []
    commission = legislator_soup.find_all('dd')
    for dlitem in commission:
        dlstring = str(dlitem.string)
        if 'Комиссия' in dlstring:
            comission = dlstring.strip().split('(')[0]
            comissions.append(comission)

    # Закидываем всё это в словарь, потому что я забыл как это делать правильно
    dictionary = {  'Name': name,
                'Faction': faction,
                'Comittee': comittee,
                'Comission':comissions}
    legislators = legislators.append(dictionary,ignore_index=True)

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_frame.py", line 945, in trace_dispatch
    self.do_wait_suspend(thread, frame, event, arg)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_frame.py", line 164, in do_wait_suspend
    self._args[0].do_wait_suspend(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [242]:
# Убираем [] из колонки, чтобы csv-шка была красивой
legislators.Comission = legislators.Comission.apply(lambda y: np.nan if len(y)==0 else y)

In [244]:
# Сохраняем файл в csv, 'denispavlenko' нужно менять на свой адрес
legislators.to_csv('/Users/denispavlenko/Documents/Masters/legislators.csv')